In [1]:
from datasets import load_dataset
from sentence_transformers import models, losses, datasets
from sentence_transformers import LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

model_save_path = 'output/matryoshka_nli_distilroberta-base-2024-02-24_13-20-43'

model = SentenceTransformer(model_save_path)

In [7]:
stsb_test = load_dataset("mteb/stsbenchmark-sts", split="test")
docs = stsb_test['sentence1']
docs = list(set(docs))
print(len(docs))

1256


In [8]:
from bunkatopics import Bunka
from langchain_community.embeddings import HuggingFaceEmbeddings

# Choose your embedding model

model_name = "nomic-ai/nomic-embed-text-v1"
#model_name = "all-MiniLM-L6-v2"
embedding_model = HuggingFaceEmbeddings(model_name=model_name,# We recommend starting with a small model
                                        model_kwargs={"device": "cuda", "trust_remote_code":True}, # Or cuda if you have GPU
                                        #encode_kwargs={"show_progress_bar": True}, # Show the progress of embeddings
                                        multi_process=False
                                        )  # set to True if you have mutliprocessing

# Initialize Bunka with your chosen model and language preference
bunka = Bunka(embedding_model=embedding_model, language='english') # You can choose any language you prefer

# Fit Bunka to your list of text
bunka.fit(docs)

You try to use a model that was created with version 2.4.0.dev0, however, your version is 2.4.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



<All keys matched successfully>
2024-02-26 13:09:24 - Bunka - INFO - Embedding documents... (can take varying amounts of time depending on their size)
2024-02-26 13:09:27 - Bunka - INFO - Reducing the dimensions of embeddings...
2024-02-26 13:09:36 - Bunka - INFO - Extracting meaningful terms from documents...
100%|██████████| 1256/1256 [00:09<00:00, 138.84it/s]


In [11]:
from sklearn.cluster import KMeans

clustering_method = KMeans(n_clusters=10, random_state=42)

bunka.get_topics(n_clusters=25, 
                    name_length=10, 
                    min_count_terms = 3, 
                    top_terms_overall = 1000000,
                    max_doc_per_topic = 2000,
                    min_docs_per_cluster = 1000,
                    ranking_terms = 15,
                    ngrams = [1,2],
                    custom_clustering_model = clustering_method
                    ) # Specify the number of terms to describe each topic

bunka.visualize_topics(width=800, height=800)

2024-02-26 13:11:12 - Bunka - INFO - Computing the topics
2024-02-26 13:11:13 - Bunka - INFO - Creating the Bunka Map


In [16]:

test_evaluator = EmbeddingSimilarityEvaluator(
    stsb_test["sentence1"],
    stsb_test["sentence2"],
    [score / 5 for score in stsb_test["score"]],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-test",
)

test_evaluator(model, output_path=model_save_path)


0.84034656072365